In [1]:

# Import Pyomo libraries
from pyomo.environ import Constraint, exp, Param, Set, units as pyunits, Var

# Import IDAES cores
from idaes.core import (
    declare_process_block_class,
    MaterialFlowBasis,
    ReactionParameterBlock,
    ReactionBlockDataBase,
    ReactionBlockBase,
)
from idaes.core.util.constants import Constants as const
import idaes.logger as idaeslog

# Set up logger
_log = idaeslog.getLogger(__name__)


In [2]:


@declare_process_block_class("NH3ReactionParameterBlock")
class NH3ReactionParameterData(ReactionParameterBlock):
    """
    Reaction Parameter Block Class
    """

    def build(self):
        """
        Callable method for Block construction.
        """
        super(NH3ReactionParameterData, self).build()

        self._reaction_block_class = NH3ReactionBlock

        # Rate Reaction Index
        self.rate_reaction_idx = Set(initialize=["R1"])

        # Rate Reaction Stoichiometry
        self.rate_reaction_stoichiometry = {
            ("R1", "Vap", "NH3"): -1,
            ("R1", "Vap", "H2"): 1.5,
            ("R1", "Vap", "N2"): 0.5,
        }

        # Equilibrium Reaction Index
        self.equilibrium_reaction_idx = Set(initialize=["E1"])

        # Equilibrium Reaction Stoichiometry
        self.equilibrium_reaction_stoichiometry = {
            ("E1", "Vap", "NH3"): -2,
            ("E1", "Vap", "H2"): 3,
            ("E1", "Vap", "N2"): 1,
        }

        # Arrhenius Constant
        self.arrhenius = Param(
            default=5e-4,
            doc="Arrhenius constant",
            units=pyunits.mol / pyunits.m**3 / pyunits.s / pyunits.Pa**2,
        )

        # Activation Energy
        self.energy_activation = Param(
            default=140000, doc="Activation energy", units=pyunits.J / pyunits.mol
        )

    @classmethod
    def define_metadata(cls, obj):
        obj.add_properties(
            {"k_rxn": {"method": None}, "reaction_rate": {"method": None}}
        )
        obj.add_default_units(
            {
                "time": pyunits.s,
                "length": pyunits.m,
                "mass": pyunits.kg,
                "amount": pyunits.mol,
                "temperature": pyunits.K,
            }
        )
        
"""Now that the various parts of the Reaction Parameter Block have been declared, 
we can assemble the actual class that will assemble these components in a flowsheet. """

class _NH3ReactionBlock(ReactionBlockBase):
    """
    This Class contains methods which should be applied to Reaction Blocks as a
    whole, rather than individual elements of indexed Reaction Blocks.
    """

    def initialize(blk, outlvl=idaeslog.NOTSET, **kwargs):
        """
        Initialization routine for reaction package.

        Keyword Arguments:
            outlvl : sets output level of initialization routine

        Returns:
            None
        """
        init_log = idaeslog.getInitLogger(blk.name, outlvl, tag="properties")
        init_log.info("Initialization Complete.")



In [3]:

@declare_process_block_class("NH3ReactionBlock", block_class=_NH3ReactionBlock)
class NH3ReactionBlockData(ReactionBlockDataBase):
    """
    reaction package for ammonia cracking
    """

    def build(self):
        """
        Callable method for Block construction
        """
        super(NH3ReactionBlockData, self).build()
        """define property variables"""
        self.k_rxn = Var(
            initialize=7e-10,
            doc="Rate constant",
            units=pyunits.mol / pyunits.m**3 / pyunits.s / pyunits.Pa**2,
        )

        self.k_eq = Param(initialize=10000, doc="Equlibrium constant", units=pyunits.Pa)

        self.reaction_rate = Var(
            self.params.rate_reaction_idx,
            initialize=0,
            doc="Rate of reaction",
            units=pyunits.mol / pyunits.m**3 / pyunits.s,
        )

        """define constraints for the rate-based reactions"""
        self.arrhenius_equation = Constraint(
            expr=self.k_rxn
            == self.params.arrhenius
            * exp(
                -self.params.energy_activation
                / (const.gas_constant * self.state_ref.temperature)
            )
        )

        def rate_rule(b, r):
            return b.reaction_rate[r] == (
            b.k_rxn
            * b.state_ref.mole_frac_comp["NH3"]**2
            * b.state_ref.pressure**2
            )

        self.rate_expression = Constraint(self.params.rate_reaction_idx, rule=rate_rule)

        """define constraints for the equilibrium-based reactions"""

        # Equilibrium constraint
        self.equilibrium_constraint = Constraint(
            expr=self.k_eq
            * self.state_ref.mole_frac_comp["N2"]
            * self.state_ref.mole_frac_comp["H2"]**3
            * self.state_ref.pressure**2
            == self.state_ref.mole_frac_comp["NH3"]**2
            * self.state_ref.pressure
        )

    def get_reaction_rate_basis(b):
        return MaterialFlowBasis.molar